In [1]:
import operator
import math
import random
import csv

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    main()

Creating pandas dataframe


,fitness_avg,fitness_std,fitness_min,fitness_max,fitness_gen,fitness_nevals,size_avg,size_std,size_min,size_max,size_gen,size_nevals
0,1.432543,3.056675,1.837110e-01,44.443656,1.0,166.0,3.606667,1.777251,1.0,12.0,1.0,166.0
1,2.168790,18.187338,1.655718e-01,315.736371,2.0,166.0,3.550000,1.625064,1.0,9.0,2.0,166.0
2,0.982550,0.712666,1.655718e-01,2.982903,3.0,163.0,3.426667,1.450732,1.0,9.0,3.0,163.0
3,0.836017,0.979399,1.655718e-01,14.538018,4.0,153.0,3.770000,1.640254,1.0,11.0,4.0,153.0
4,0.944635,1.616144,1.655718e-01,18.973920,5.0,158.0,3.776667,1.628943,1.0,10.0,5.0,158.0
5,0.885819,1.002961,1.655718e-01,14.218070,6.0,169.0,4.000000,1.876166,1.0,10.0,6.0,169.0
6,0.731332,0.560160,1.655718e-01,3.352923,7.0,167.0,4.350000,1.927217,1.0,10.0,7.0,167.0
7,0.785916,1.242598,1.300804e-01,19.185174,8.0,187.0,5.136667,2.214646,1.0,14.0,8.0,187.0
8,0.672788,1.007641,1.237193e-01,14.233920,9.0,176.0,5.716667,2.483893,1.0,14.0,9.0,176.0
9,0.786975,2.566789,1.237193e-01,32.495152,10.0,176.0,6.273333,2.339221,1.0,15.0,10.0,176.0
